In [1]:
!pip install transformers

In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
#import plotly.express as px
import tensorflow as tf
import pandas as pd
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [3]:
train_df = pd.read_csv('train.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test_df = pd.read_csv('test.csv')
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
#Procesamiento de datos
X_train, X_test, y_train, y_test = train_test_split(train_df['text'], train_df['target'], test_size=0.33,
                                                  random_state=42)
FTest = test_df['text']
test_ids = test_df['id'] 

In [ ]:
#Parametros
vocab_size =60522
epochs = 3
maxlen = 60
n_words = 500
embedding_dim = 16

In [7]:
#Tokenización
#tokenizer = Tokenizer(num_words = n_words, oov_token = '<OOV>')

#tokenizer.fit_on_texts(X_train)
#word_index = tokenizer.word_index

#sequences_train = tokenizer.texts_to_sequences(X_train)
#sequences_test = tokenizer.texts_to_sequences(X_test)
#sequences_FTest = tokenizer.texts_to_sequences(FTest)




#padded_train = pad_sequences(sequences_train, maxlen = maxlen, truncating = 'post')
#padded_test = pad_sequences(sequences_test, maxlen = maxlen)
#padded_FTest = pad_sequences(sequences_FTest, maxlen = maxlen)

In [8]:
def tokenizar(textos):
    secuencia = []
    for text in textos:
        marked_text = "[CLS] " + text + " [SEP]"
        tokenized_text = tokenizer.tokenize(marked_text)
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
        secuencia.append(indexed_tokens)
    return secuencia


In [9]:
secuencia_train = tokenizar(X_train.values)
secuencia_test = tokenizar(X_test.values)
secuencia_FTest = tokenizar(FTest.values)
padded_train = pad_sequences(secuencia_train, maxlen = maxlen, truncating = 'post')
padded_test = pad_sequences(secuencia_test, maxlen = maxlen)
padded_FTest = pad_sequences(secuencia_FTest, maxlen = maxlen)

In [10]:
type(padded_train)

numpy.ndarray

In [11]:
len(secuencia_FTest[125])

43

In [12]:
#modelo LSTM
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          1953408   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                33024     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 1,990,657
Trainable params: 1,990,657
Non-trainable params: 0
_________________________________________________________________


In [13]:
#entreno
history_lstm = model.fit(padded_train, y_train, epochs=epochs, validation_data = (padded_test,y_test))

Epoch 1/7
160/160 [==============================] - 65s 405ms/step - loss: 0.6030 - accuracy: 0.6698 - val_loss: 0.4782 - val_accuracy: 0.7927
Epoch 2/7
160/160 [==============================] - 65s 405ms/step - loss: 0.3416 - accuracy: 0.8565 - val_loss: 0.4867 - val_accuracy: 0.7752
Epoch 3/7
160/160 [==============================] - 65s 404ms/step - loss: 0.1923 - accuracy: 0.9290 - val_loss: 0.5808 - val_accuracy: 0.7680
Epoch 4/7
160/160 [==============================] - 66s 410ms/step - loss: 0.1170 - accuracy: 0.9586 - val_loss: 0.7083 - val_accuracy: 0.7589
Epoch 5/7
160/160 [==============================] - 65s 405ms/step - loss: 0.0750 - accuracy: 0.9749 - val_loss: 0.8855 - val_accuracy: 0.7672
Epoch 6/7
160/160 [==============================] - 67s 417ms/step - loss: 0.0472 - accuracy: 0.9839 - val_loss: 0.8942 - val_accuracy: 0.7433
Epoch 7/7
160/160 [==============================] - 65s 404ms/step - loss: 0.0346 - accuracy: 0.9873 - val_loss: 1.0781 - val_accuracy:

In [14]:
preds = model.predict_classes(padded_FTest)
#preds = pd.Series(preds)
preds = pd.Series(list((x[0] for x in preds)))
preds
df_preds = pd.concat([test_ids,preds],axis=1)
df_preds.rename(columns = {0 : 'target'}, inplace=True)
df_preds.set_index('id', inplace=True)
df_preds.to_csv('BERT-LSTM.csv')
df_preds.head()

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


,target
id,
0,0
2,1
3,1
9,1
11,1


In [15]:
#modelo LSTM bidireccional
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64, input_length=maxlen),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 60, 64)            1953408   
_________________________________________________________________
bidirectional (Bidirectional (None, 60, 128)           66048     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 2,064,897
Trainable params: 2,064,897
Non-trainable params: 0
_________________________________________________________________


In [16]:
history_bi = model.fit(padded_train, y_train, epochs=epochs, validation_data=(padded_test, y_test))

Epoch 1/7
160/160 [==============================] - 256s 2s/step - loss: 0.6086 - accuracy: 0.6604 - val_loss: 0.5198 - val_accuracy: 0.7529
Epoch 2/7
160/160 [==============================] - 250s 2s/step - loss: 0.3677 - accuracy: 0.8502 - val_loss: 0.5060 - val_accuracy: 0.7660
Epoch 3/7
160/160 [==============================] - 248s 2s/step - loss: 0.2168 - accuracy: 0.9224 - val_loss: 0.6001 - val_accuracy: 0.7692
Epoch 4/7
160/160 [==============================] - 310s 2s/step - loss: 0.1192 - accuracy: 0.9604 - val_loss: 0.7417 - val_accuracy: 0.7565
Epoch 5/7
160/160 [==============================] - 226s 1s/step - loss: 0.0917 - accuracy: 0.9706 - val_loss: 0.8744 - val_accuracy: 0.7417
Epoch 6/7
160/160 [==============================] - 227s 1s/step - loss: 0.0611 - accuracy: 0.9825 - val_loss: 0.8801 - val_accuracy: 0.7405
Epoch 7/7
160/160 [==============================] - 231s 1s/step - loss: 0.0454 - accuracy: 0.9886 - val_loss: 0.9545 - val_accuracy: 0.7525


In [17]:
preds = model.predict_classes(padded_FTest)
#preds = pd.Series(preds)
preds = pd.Series(list((x[0] for x in preds)))
preds
df_preds = pd.concat([test_ids,preds],axis=1)
df_preds.rename(columns = {0 : 'target'}, inplace=True)
df_preds.set_index('id', inplace=True)
df_preds.to_csv('BERT-BiLSTM.csv')
df_preds.head()

KeyboardInterrupt: 

In [20]:
#modelo Conv1D
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64, input_length=maxlen),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 60, 64)            1953408   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 56, 128)           41088     
_________________________________________________________________
global_average_pooling1d_1 ( (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 2,002,817
Trainable params: 2,002,817
Non-trainable params: 0
_________________________________________________________________


In [21]:
history_conv = model.fit(padded_train, y_train, epochs=epochs, validation_data=(padded_test, y_test))

Epoch 1/7
160/160 [==============================] - 120s 750ms/step - loss: 0.5875 - accuracy: 0.6794 - val_loss: 0.4620 - val_accuracy: 0.7895
Epoch 2/7
160/160 [==============================] - 109s 684ms/step - loss: 0.3402 - accuracy: 0.8582 - val_loss: 0.4592 - val_accuracy: 0.8022
Epoch 3/7
160/160 [==============================] - 112s 702ms/step - loss: 0.1998 - accuracy: 0.9286 - val_loss: 0.5511 - val_accuracy: 0.7903
Epoch 4/7
160/160 [==============================] - 122s 760ms/step - loss: 0.1145 - accuracy: 0.9608 - val_loss: 0.6865 - val_accuracy: 0.7573
Epoch 5/7
160/160 [==============================] - 99s 616ms/step - loss: 0.0680 - accuracy: 0.9784 - val_loss: 0.8158 - val_accuracy: 0.7636
Epoch 6/7
160/160 [==============================] - 107s 667ms/step - loss: 0.0452 - accuracy: 0.9875 - val_loss: 0.9282 - val_accuracy: 0.7604
Epoch 7/7
160/160 [==============================] - 116s 727ms/step - loss: 0.0325 - accuracy: 0.9884 - val_loss: 1.0968 - val_acc

In [ ]:
preds = model.predict_classes(padded_FTest)
#preds = pd.Series(preds)
preds = pd.Series(list((x[0] for x in preds)))
preds
df_preds = pd.concat([test_ids,preds],axis=1)
df_preds.rename(columns = {0 : 'target'}, inplace=True)
df_preds.set_index('id', inplace=True)
df_preds.to_csv('BERT-Conv1D.csv')
df_preds.head()

In [22]:
#modelo GRU
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 16)          488352    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 64)                9600      
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 390       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 7         
Total params: 498,349
Trainable params: 498,349
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history_gru = model.fit(padded_train, y_train, epochs=epochs, validation_data=(padded_test, y_test))

Epoch 1/7


In [ ]:
import plotly.graph_objects as go
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("LSTM", "Bi LSTM", "Convolution", "GRU"),
    shared_yaxes=True,
    shared_xaxes=True,
    vertical_spacing=0.1,
    horizontal_spacing=0.03,
    x_title='epochs',
    y_title='val_accuracy'
)

epochs = list(range(1, len(history_lstm.history['accuracy'])+1))

fig.add_trace(
    go.Scatter(x=epochs,, y=history_lstm.history['accuracy'], name='accuracy', text='LSTM'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=epochs, y=history_lstm.history['val_accuracy'], name='val_accuracy'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=epochs, y=history_bi.history['accuracy'], name='accuracy'),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=epochs, y=history_bi.history['val_accuracy'], name='val_accuracy'),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(x=epochs, y=history_conv.history['accuracy'], name='accuracy'),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=epochs, y=history_conv.history['val_accuracy'], name='val_accuracy'),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=epochs, y=history_gru.history['accuracy'], name='accuracy'),
    row=2, col=2
)
fig.add_trace(
    go.Scatter(x=epochs, y=history_gru.history['val_accuracy'], name='val_accuracy'),
    row=2, col=2
)

fig.update_layout(title='Accuracy', showlegend=False)


In [ ]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("LSTM", "Bi LSTM", "Convolution", "GRU"),
    shared_yaxes=True,
    shared_xaxes=True,
    vertical_spacing=0.1,
    horizontal_spacing=0.03)

epochs = list(range(1, len(history_lstm.history['loss'])+1))

fig.add_trace(
    go.Scatter(x=epochs, y=history_lstm.history['loss'], name='loss', text='LSTM'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=epochs, y=history_lstm.history['val_loss'], name='val_loss'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=epochs, y=history_bi.history['loss'], name='loss'),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=epochs, y=history_bi.history['val_loss'], name='val_loss'),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(x=epochs, y=history_conv.history['loss'], name='loss'),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=epochs, y=history_conv.history['val_loss'], name='val_loss'),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=epochs, y=history_gru.history['loss'], name='loss'),
    row=2, col=2
)
fig.add_trace(
    go.Scatter(x=epochs, y=history_gru.history['val_loss'], name='val_loss'),
    row=2, col=2
)

fig.update_layout(title='Loss', showlegend=False)

In [ ]:
preds = model.predict_classes(padded_FTest)
#preds = pd.Series(preds)
preds = pd.Series(list((x[0] for x in preds)))
preds

In [ ]:
df_preds = pd.concat([test_ids,preds],axis=1)
df_preds.rename(columns = {0 : 'target'}, inplace=True)
df_preds.set_index('id', inplace=True)
df_preds.to_csv('BERT-GRU.csv')
df_preds.head()

Tuning: 
1-  epochs = 10
    maxlen = 50
    n_words = 500
    embedding_dim = 16
Acuracy: 0.7441

2-  epochs = 6
    maxlen = 40
    n_words = 500
    embedding_dim = 16
Accuracy : 0.7656b

3-  epochs = 5
    maxlen = 40
    n_words = 300
    embedding_dim = 16
Accuracy: 0.7437
s